In [13]:
import os
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.parse import urlencode
import requests

In [14]:
API_KEY = "a1f5eae4966a5e0e09ec7d83b2f3819e"
COLUMN_NAMES = ['titulo', 'preço', 'condição', 'qntd vendida', 'marca', 'linha', 'modelo', 'numero_peça', 'qntd_velas', 'url']

In [9]:
def get_scraperapi_url(url):
    """
        Converts url into API request for ScraperAPI.
    """
    payload = {'api_key': API_KEY, 'url': url}
    proxy_url = 'http://api.scraperapi.com/?' + urlencode(payload)
    return proxy_url

# Unir relatorios da fase 1

In [ ]:
df = pd.DataFrame(columns=COLUMN_NAMES)
path = './relatorios/'
files = os.listdir(path)

In [ ]:
tamanho = len(files)

for f in files:
    print("Falta: ", str(tamanho))
    df_aux = pd.read_csv(path + f, index_col=0)
    df = df.append(df_aux, ignore_index=True)
    tamanho -= 1

# Separar os que nao tem titulo

In [ ]:
df_anuncios_sem_titulo = df[df["titulo"] == "Não tem titulo"]

In [ ]:
# salvar apenas url
df_anuncios_sem_titulo = df_anuncios_sem_titulo[['url']]

# mudar nome da coluna
df_anuncios_sem_titulo.columns = ['urls']

In [ ]:
# total de urls
len(df_anuncios_sem_titulo)

In [ ]:
# restart index
df_anuncios_sem_titulo = df_anuncios_sem_titulo.reset_index(drop=True)

## Separar em df de 100 url

In [ ]:
list_df = []
n = 10
for i in range(0,df_anuncios_sem_titulo.shape[0],n):
    list_df.append(df_anuncios_sem_titulo[i:i+n])

In [ ]:
# total de df
len(list_df)

## Separar os dfs em 10 grupos  

In [ ]:
# dividir a lista de df em 10 grupos
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
grupos_df = list(chunks(list_df, round(len(list_df)/10)))

In [ ]:
len(grupos_df)

## Salvar nas 10 pastas

In [ ]:
aux_n = 0
for i in range(1, 11):
    n = round(len(list_df)/10)
    grupo_df = list_df[aux_n:aux_n + n]
    
    name_aux = 1
    for df_save in grupo_df:
        df_save.to_csv('./bases/bases'+str(i)+'/anuncios_sem_titulo_'+str(i)+"_"+str(name_aux)+'.csv', index=False)
        name_aux += 1
    
    aux_n += n

# Salvar os que estão certos

In [ ]:
df_anuncios_com_titulo = df[df["titulo"] != "Não tem titulo"]

In [ ]:
df_anuncios_com_titulo.to_csv('df1.csv', index=False)

# Unir e salvar os links que forem refeitos

In [ ]:
path = './relatorios/relatorios_novos/'
files = os.listdir(path)

In [ ]:
df = pd.DataFrame(columns=COLUMN_NAMES)

In [ ]:
tamanho = len(files)

for f in files:
    print("Falta: ", str(tamanho))
    df_aux = pd.read_csv(path + f, index_col=0)
    df = df.append(df_aux, ignore_index=True)
    tamanho -= 1

In [ ]:
df_anuncios_com_titulo = df[df["titulo"] != "Não tem titulo"]
df_anuncios_com_titulo.to_csv('df2.csv', index=False)

# Unir os 2 CSVs

In [3]:
df1 = pd.read_csv('df1.csv')
df2 = pd.read_csv('df2.csv')

c:\users\hiago bonamelli\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df = pd.DataFrame(columns=COLUMN_NAMES)

In [5]:
df = df.append(df1)
df = df.append(df2)

# Tratar base

## Tirar anúncios sem venda

In [6]:
df = df[df["qntd vendida"] != "Não tem qntd de vendas"]

## Mudar tipo coluna quantidade vendida pra número

In [7]:
df["qntd vendida"] = df["qntd vendida"].astype(int)

## Mudar coluna preço

### Pegar todos os preços faltando

In [31]:
len(df[df["preço"] == "não tem preço"])

1

In [33]:
for index, row in df.iterrows():
    if row["preço"] == "não tem preço":
        print(row["url"])
        page = requests.get(row["url"])
        soup = BeautifulSoup(page.content, 'html.parser')
        
        # preco
        div_preco = soup.find_all("div", class_="ui-pdp-price__second-line")[0]
        fraction = div_preco.find_all("span", class_="price-tag-fraction")[0].text
        try:
            cents = div_preco.find_all("span", class_="price-tag-cents")[0].text
        except:
            cents = "0"
        preco = "R$" + fraction + "." + cents
        df.at[index, 'preço'] = preco
        print(preco)
        time.sleep(3)

https://produto.mercadolivre.com.br/MLB-1916402432-jogo-vela-aquecedora-frontier-sel-25-diesel-2007-a-2012-_JM#position=32&search_layout=stack&type=item&tracking_id=5afc3325-e13e-4e82-86b0-cc6967458eba
R$316.0


In [15]:
page = requests.get(get_scraperapi_url("https://produto.mercadolivre.com.br/MLB-1488650291-chave-vela-articulada-16mm21mm-automotiva-veicular-_JM#position=6&search_layout=stack&type=item&tracking_id=a6760de9-35c7-4660-b935-347bbeae1032"))
soup = BeautifulSoup(page.content, 'html.parser')
fraction = soup.find_all("span", class_='price-tag-fraction')[0].text
cents = soup.find_all("span", class_="price-tag-cents")[0].text
preco = "R$" + fraction + "." + cents

In [16]:
preco

'R$29.90'

In [ ]:
page = requests.get("https://produto.mercadolivre.com.br/MLB-1325010005-jogo-de-velas-citroen-c4-pallas-20-16v-flex-marelli-kh6rfmm-_JM#position=4&search_layout=stack&type=item&tracking_id=999ca883-cd97-489b-820e-6aecf40262c6")
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
div_preco = soup.find_all("div", class_="ui-pdp-price__second-line")[0]

In [ ]:
div_preco.find_all("span", class_="price-tag-fraction")[0].text


In [ ]:
div_preco.find_all("span", class_="price-tag-cents")[0].text

In [ ]:
contagem = 0
for index, row in df.iterrows():
    if row["preço"] == "não tem preço":
        contagem += 1
    
print(contagem)

In [34]:
df.to_excel('df_final.xlsx')